In [1]:
pip install deepface

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install tf-keras

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
from deepface import DeepFace
import os

Krok číslo 1.
 - definování cesty k adresáří s emocemi path =  C:\Users\zitao\Documents\PETR\DATA SCIENCE\EMOCE\train
 - definování cesty k adresáři s konkrétními emocemi
 - definování cesty ke kontrétnimu souboru/fotce
 - zapisování název souboru, pokud výsledek analýzy emoce z fotky bude odpovídat názvu složky, který právě zkoumáne do seznamu vhodných souborů ( fotku, kterou chceme)
    (seznam_fotek_OK) 
 - zapisování název souboru, pokud výsledek analýzy emoce z fotky nebude odpovídat názvu složky, který právě zkoumáne do seznamu nevhodných souborů ( fotku, kterou nechceme)
    (seznam_fotek_NOK)
 - zapisování název souboru, pokud není schopen vůbec analyzovat emoce z fotky nebo soubor nahrát
    (chybny_soubor)
 - opakuj pro každý soubor v každé složce -->

Krok číslo 2.
 - přesunování nevhodné fotky, pokud výsledek analýzy emoce z fotky nebude odpovídat názvu složky
 - rozdělení NOK a OK fotky pro každou složku

# Definice cesty k složkám 

In [16]:
path = r"C:\Users\zitao\Documents\PETR\DATA_SCIENCE\EMOCE"

In [17]:
## Seznam složek v adresářích
fotky_emoce = os.listdir(f"{path}/train")
fotky_emoce

## seznam_souboru = os.listdir(f"{path}/train/{nazev_slozky}")

['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

## Analýza obrázku

In [18]:
seznam_fotek_NOK = []
seznam_fotek_OK = []
chybny_soubor = []

In [19]:
for i in range (0,len(fotky_emoce)): 
    nazev_slozky = fotky_emoce [i]
    cesta_k_emocim = (f"{path}/train/{nazev_slozky}")

    zdrojova_slozka = cesta_k_emocim 
    cilova_slozka =  (f"{path}/train_OK/{nazev_slozky}")
    
    # Vytvoření cílové složky, pokud neexistuje
    os.makedirs(cilova_slozka, exist_ok=True)
    
     
    for obrazek1 in os.listdir(cesta_k_emocim):  
        cesta_obrazku = os.path.join(cesta_k_emocim, obrazek1)  
        obrazek2 = cv2.imread(cesta_obrazku)  
    
        if obrazek2 is None:
            ## print(f"Chyba: Obrázek {obrazek1} se nepodařilo načíst.")
            chybny_soubor.append(obrazek1)  # Uložit název souboru, ne None
            continue  
    
        try:
            prediction = DeepFace.analyze(obrazek2, actions=['emotion'], enforce_detection=True)  ## TRUE pokud není na obrázku detekován žádný obličej, vyvolá se chyba
            emoce = prediction[0]['dominant_emotion']
    
            if emoce == (nazev_slozky):
                seznam_fotek_OK.append(cesta_obrazku)

            # # Kopírování souborů pomocí os.system (závislé na OS)
                cilova_cesta = os.path.join(cilova_slozka, obrazek1)
                os.system(f'copy "{cesta_obrazku}" "{cilova_cesta}"')
                    
                    # if os.path.isfile(plna_cesta):  # Ověření, že jde o soubor
                    #      if os.name == "nt":  # Windows
                    #         os.system(f'copy "{plna_cesta}" "{cilova_cesta}"')
                    #      else:  # Linux/macOS
                    #         os.system(f'cp "{plna_cesta}" "{cilova_cesta}"')
                    #      print(f"Zkopírován: {soubor}")
            
            else:
                seznam_fotek_NOK.append(cesta_obrazku)
    
        except ValueError as e:
            ## print(f"Chyba: DeepFace nemohl analyzovat {cesta_obrazku} – {e}")
            chybny_soubor.append(cesta_obrazku)  # Uložit problematický soubor
        except Exception as e:
           ## print(f"Neočekávaná chyba u {cesta_obrazku}: {e}")
            chybny_soubor.append(cesta_obrazku)  # Uložit problematický soubor

KeyboardInterrupt: 

In [8]:
len(seznam_fotek_OK)   

11302

In [20]:
len(seznam_fotek_NOK)

4916

In [21]:
len(chybny_soubor)

10642

In [22]:
## seznam_fotek_NOK

In [12]:
## chybny_soubor

In [13]:
##seznam_fotek_OK

In [ ]:
import os
import cv2
import shutil
from deepface import DeepFace

for nazev_slozky in fotky_emoce:
    cesta_k_emocim = f"{path}/train/{nazev_slozky}"
    cilova_slozka = f"{path}/train_OK_2/{nazev_slozky}"
    
    os.makedirs(cilova_slozka, exist_ok=True)  # Vytvoření cílové složky, pokud neexistuje
    
    for obrazek in os.listdir(cesta_k_emocim):  
        cesta_obrazku = os.path.join(cesta_k_emocim, obrazek)
        obrazek_data = cv2.imread(cesta_obrazku)
    
        if obrazek_data is None:
            chybny_soubor.append(cesta_obrazku)
            continue  
    
        try:
            prediction = DeepFace.analyze(obrazek_data, actions=['emotion'], enforce_detection=True)
            emoce = prediction[0]['dominant_emotion']
    
            if emoce == nazev_slozky:
                seznam_fotek_OK.append(cesta_obrazku)
                shutil.copy(cesta_obrazku, os.path.join(cilova_slozka, obrazek))  # Kopírování souboru
            
            else:
                seznam_fotek_NOK.append(cesta_obrazku)
    
        except Exception as e:
            chybny_soubor.append(cesta_obrazku)
            print(f"Chyba u {cesta_obrazku}: {e}")  # Logování chyby